# Tabular Datada Keras Tuner ve Feature Space

## -TF 2.12 ve üstü gereklidir-

In [ ]:
!nvidia-smi

Sun Apr 30 23:37:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import keras_tuner
from tensorflow.keras.utils import FeatureSpace

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split as tts

tf.__version__ # 2.12

'2.12.0'

In [ ]:
# Data linki: https://www.kaggle.com/competitions/flight-delays-fall-2018/overview
df = pd.read_csv("/content/flight_delays_train.csv")

df['dep_delayed_15min'] = np.where(
                            df['dep_delayed_15min'] == 'Y', 1, 0)

df.dropna(inplace=True, axis=0)

df.sample(10)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
44645,c-11,c-29,c-2,2015,DL,ATL,MEM,332,0
16547,c-7,c-19,c-3,1922,CO,EWR,IAH,1400,1
33779,c-9,c-26,c-1,1658,AA,BOS,ORD,867,0
69114,c-6,c-10,c-5,1540,OO,SLC,FCA,532,0
81360,c-3,c-27,c-7,1531,NW,DTW,LAN,74,0
40039,c-11,c-25,c-5,645,OH,OMA,CVG,614,0
57816,c-2,c-2,c-3,1245,WN,HOU,STL,687,0
93615,c-3,c-10,c-5,1823,FL,ATL,LAS,1747,0
66204,c-9,c-3,c-6,1123,WN,MDW,IND,162,0
49056,c-6,c-5,c-7,1237,OO,SMF,LAX,373,1


In [ ]:
df["dep_delayed_15min"].value_counts()

0    80956
1    19044
Name: dep_delayed_15min, dtype: int64

## Datasetleri Oluşturma

- Amacım pipeline kurmayı göstermek olduğu için EDA kısmını atlıyorum.

In [ ]:
train_dataframe, valid_dataframe = tts(df, 
                                       train_size=0.8,
                                       random_state=100)

In [ ]:
valid_dataframe["dep_delayed_15min"].value_counts(normalize=True)

0    0.8077
1    0.1923
Name: dep_delayed_15min, dtype: float64

In [ ]:
train_dataframe["dep_delayed_15min"].value_counts(normalize=True)

0    0.810025
1    0.189975
Name: dep_delayed_15min, dtype: float64

In [ ]:
feature_type_mapping = {
        "Month": "string_categorical",
        "DayofMonth": "string_categorical",
        "DayOfWeek": "string_categorical",
        "UniqueCarrier": "string_categorical",
        "Origin": "string_categorical",
        "Dest": "string_categorical",
        "DepTime": "float_normalized",
        "Distance": "float_normalized",
    }

In [ ]:
def replace_rare_categories(train_data, val_data, columns, threshold=30):
    """
    Eğitim verilerinde belirtilen eşik değerinden daha az görünen kategorileri
    'Other' adlı yeni bir kategoriyle değiştirir.
    
    Parametreler
    ----------
    train_data : pd.DataFrame
        Eğitim veri seti.
    val_data : pd.DataFrame
        Doğrulama veri seti.
    columns : dict
        Değiştirilecek sütunların adlarını ve türlerini içeren sözlük.
    threshold : int, optional (default=30)
        Kategorilerin nadir sayılması için kullanılacak eşik değeri.

    Returns
    -------
    train_data : pd.DataFrame
        Kategorileri değiştirilmiş eğitim veri seti.
    val_data : pd.DataFrame
        Kategorileri değiştirilmiş doğrulama veri seti.
    replaced_categories : dict
        Değiştirilen nadir kategorileri içeren sözlük.
    """
    replaced_categories = {}
    
    for column, column_type in columns.items():
        if column_type == "string_categorical":
            print(f"Column {column} değerleri kontrol ediliyor")
            
            value_counts = train_data[column].value_counts()
            rare_categories = value_counts.iloc[threshold:].index
            
            if len(rare_categories):
                print(f"\tNadir kategoriler --> {rare_categories}")
                replaced_categories[column] = rare_categories
                
                train_data[column] = train_data[column].replace(rare_categories,
                                                                "Other")
                val_data[column] = val_data[column].replace(rare_categories, 
                                                            "Other")
    
    return train_data, val_data, replaced_categories

train_dataframe, valid_dataframe, replaced_categories = replace_rare_categories(
                                                        train_dataframe,
                                                        valid_dataframe,
                                                        feature_type_mapping,
                                                        threshold=100)

Column Month değerleri kontrol ediliyor
Column DayofMonth değerleri kontrol ediliyor
Column DayOfWeek değerleri kontrol ediliyor
Column UniqueCarrier değerleri kontrol ediliyor
Column Origin değerleri kontrol ediliyor
	Nadir kategoriler --> Index(['KOA', 'MSN', 'LBB', 'JAN', 'CAK', 'MDT', 'LEX', 'MAF', 'SGF', 'AMA',
       ...
       'TEX', 'RFD', 'AKN', 'ADK', 'BLI', 'GST', 'WYS', 'VCT', 'VIS', 'OAJ'],
      dtype='object', length=188)
Column Dest değerleri kontrol ediliyor
	Nadir kategoriler --> Index(['CAE', 'LBB', 'KOA', 'CAK', 'MSN', 'AMA', 'CRP', 'MDT', 'CID', 'SRQ',
       ...
       'SOP', 'GST', 'BLI', 'DLG', 'ACK', 'VCT', 'LWB', 'TTN', 'TEX', 'WYS'],
      dtype='object', length=189)


In [ ]:
train_dataframe["Origin"].value_counts()

Other    6360
ATL      4671
ORD      3855
DFW      3387
LAX      2639
         ... 
HSV       131
BTR       125
LGB       124
LIH       121
PNS       121
Name: Origin, Length: 101, dtype: int64

In [ ]:
def dataframe_to_tf_data(dataframe: pd.DataFrame,
                         shuffle: bool,
                         target: str) -> tf.data.Dataset:
    """
    Verilen pandas DataFrame'ini TensorFlow Dataset'e dönüştürür.
    
    Parametreler
    ----------
    dataframe : pd.DataFrame
        Dönüştürülecek pandas DataFrame.
    shuffle : bool
        Veri setini karıştırma seçeneği. Eğer True ise, veri seti karıştırılır.
    target : str
        Hedef sütunun adı.
        
    Returns
    -------
    ds : tf.data.Dataset
        Dönüştürülmüş TensorFlow Dataset.

    """
    copy_df = dataframe.copy()
    labels = copy_df.pop(target)
    
    ds = tf.data.Dataset.from_tensor_slices((dict(copy_df), 
                                             labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(copy_df))
    
    return ds

train_ds = dataframe_to_tf_data(train_dataframe, 
                                shuffle=True,
                                target="dep_delayed_15min")

valid_ds = dataframe_to_tf_data(valid_dataframe, 
                                shuffle=False,
                                target="dep_delayed_15min")

## Feature Space Oluşturma :o

```python
feature_type_mapping = {
        "Month": "string_categorical",
        "DayofMonth": "string_categorical",
        "DayOfWeek": "string_categorical",
        "UniqueCarrier": "string_categorical",
        "Origin": "string_categorical",
        "Dest": "string_categorical",
        "DepTime": "float_normalized",
        "Distance": "float_normalized",
    }
```

`FeatureSpace` kullanmak için bir dictionary kullanmamız gerekiyor, mevcut değerler şunlardan biri olabilir:
 * `integer_categorical`
 * `string_categorical`
 * `float_discretized` --> numeric olup, discretize edilir. Mesela yaş diye bir column'unuz varsa bunun içine girecektir.
 * `float_normalized` --> normalize edilecek numeric columnlar

 
 
 Varsayılan olarak kategorik columnlar one-hot-encoding şekline getirilir.

In [ ]:
feature_space = FeatureSpace(
    features=feature_type_mapping,
    
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/HashedCrossing
    # crossing kısmı için yukarıda layer ve çıktıları inceleyebilirsiniz.
    # Kısaca verdiğimiz (şuanda sadece kategorik verdik) columnlardan yeni
    # özellikler oluşturuyor.
    crosses=[("Month", "DayOfWeek"), 
             ("Origin", "Dest"),
             ("UniqueCarrier", "Origin")],
    
    # Hash uzayının boyutu 64 olacak.
    crossing_dim=64,
    
    # FeatureSpace classı bizim için bulunan özellik kadar input oluşturacak.
    # Mesela 8 column varsa, 8 input olacak ve bunlar farklı şekilde işlenecek.
    # Kategorikler OHE uygulanacak, numericler normalize edilecek. Bunlar ayrı
    # ayrı input olduğu için (crossingler mevcut inputlardan türeyecek) en sonunda
    # concat edilecek.
    # Örn: OHE yapılmış columndan (None, 20) shape ve normalize edilmiş numeric
    # column (None, 1) --> concat sonucu shape (None, 21) olacaktır.

    # Bu aşamada oluşan input layerlar:
    #   {'Month': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Month')>,
    #  'DayofMonth': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'DayofMonth')>,
    #  'DayOfWeek': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'DayOfWeek')>,
    #  'UniqueCarrier': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'UniqueCarrier')>,
    #  'Origin': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Origin')>,
    #  'Dest': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Dest')>,
    #  'DepTime': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'DepTime')>,
    #  'Distance': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Distance')>}

    output_mode="concat",
)

In [ ]:
# tf.data içinde labelları barındırdığı için ilk başta labelları olmayan train 
# verisini oluşturalım.
train_ds_with_no_labels = train_ds.batch(64).map(lambda x, _: x)

# Preprocessing layerlarında adapt methodu bulunur. FeatureSpace class'ı da bu
# methodu bulundurur. Değerleri öğrenmek için train datasını tarayıp, değerleri saklar.
# Mesela normalizasyon işlemi için train datasındaki mean ve varyansı öğrenir.
# Kategorik columnlar da aynı şekilde bir işleme mevcuttur.
feature_space.adapt(train_ds_with_no_labels)

In [ ]:
# 474 column olmuş, OHE'den dolayı da fazla column oldu. Veya kategorikleri daha iyi bir şekilde
# işlemek isterseniz, integer şekilde encode edip, Embedding layerına verip vektörler halinde
# kullanmayı deneyebilirsiniz.
for x, _ in train_ds.batch(32).take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

preprocessed_x.shape: (32, 474)
preprocessed_x.dtype: <dtype: 'float32'>


In [ ]:
# Preprocessing aşamasını isterseniz modele gömersiniz veya tf.data'ya map 
# kullanarak uygulayabilirsiniz. Bu layerları modele nasıl atacağınızı aşağıda
# açıklayacağım.

# Batch - Map --> Vektörize operasyonlar

train_ds = train_ds.batch(256).map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.batch(256).map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.cache().prefetch(tf.data.AUTOTUNE)

## Keras Tuner Kurmak

In [ ]:
def build_model(hp):
    """
    Hiperparametre araması için bir Keras modeli oluşturur.

    Parametreler
    ----------
    hp : kerastuner.HyperParameters
        Keras Tuner için hiperparametreler nesnesi.

    Returns
    -------
    model : tf.keras.Model
        Tune edilmiş parametrelerle oluşturulmuş Keras modeli.
    """
    
    encoded_features = feature_space.get_encoded_features()
    # <KerasTensor: shape=(None, 474) dtype=float32 (created by layer 'concatenate')>

    x = encoded_features
    # Convolution layer eklemek isterseniz bu şekilde yapmanız gerekiyor.
    # x = tf.keras.layers.Lambda kısmını silmeyin! Conv1D için shape 3D (batch size dahil) olmalı
    # Alttaki conv layer kısmını altta yaptığım gibi for döngüsüne alabilirsiniz
    # fakat GlobalPooling eklemeyi unutmayın. Flatten layer da olabilir.
    
    # x = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(x)

    # x = tf.keras.layers.Conv1D(
    #         filters=hp.Int(f"filters_cnn", 8, 32, step=4), 
    #         activation=hp.Choice("activation_cnn", ["relu", 
    #                                             "elu",
    #                                             "swish"]),
    #         kernel_size=3, padding="same")(x)
    # x = tf.keras.layers.MaxPool1D(pool_size = hp.Int(f"pool_size",
    #                                                  2, 4, step=1))(x)

    # x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # Kaç tane MLP layerı olsun?
    for i in range(hp.Int("mlp_layers", 1, 5)):

        x = tf.keras.layers.Dense(
            # Her layerda farklı unitler olabilir.
            units=hp.Int(f"units_{i}", 32, 256, step=32), 

            # Aktivasyonlar aynı fakat değiştirilebilir.
            activation=hp.Choice("activation_dense", ["relu", 
                                                "elu",
                                                "swish"]),
        )(x)
        # Her layer sonrası farklı dropout rate olabilir.
        x = tf.keras.layers.Dropout(hp.Float(f'dropout_{i}', 0.01, 0.5,
                                                sampling='log'))(x)

    # x = tf.keras.layers.Dropout(hp.Float('dropout_general', 0.1, 0.5,
    #                                      sampling='log'))(x)

    # Binary classification olduğu için sigmoid aktivasyonu
    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

    # Modelde şuan encoding aşaması vb yok.
    model = tf.keras.Model(inputs=encoded_features, outputs=outputs)

    # Yukarıda dediğim gibi dahil etmek isterseniz:
    # model = tf.keras.Model(inputs = feature_space.get_inputs(), outputs=outputs)
    # Fakat bu sefer datasetlerin işlemesinde hata alabilirsiniz.
    # Bu modeli daha çok productiona vermek için kullanabilirsiniz, inputları
    # dict şekilde verdiğinizde kendisi işleyecektir. Bunu daha detaylı olarak
    # farklı notebooklarda ele alacağım.

    # Farklı optimizer ve LR seçimleri deneyelim.
    optimizer_name = hp.Choice("optimizer", ["adam", "rmsprop"])
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')

    # Seçilen değerleri optimizerlara pass etmek için.
    optimizer_dict = {
        'adam': tf.keras.optimizers.Adam,
        'rmsprop': tf.keras.optimizers.RMSprop
    }

    optimizer = optimizer_dict[optimizer_name](learning_rate=learning_rate)
    
    # İki farklı loss deneyelim, BinaryFocalCrossentropy çok dengesiz verisetlerinde
    # kullanılan bir loss fonksiyonudur.
    loss_name = hp.Choice("loss", ["binary_crossentropy",
                                   "binary_focal_crossentropy"])

    loss_dict = {
        'binary_crossentropy': tf.keras.losses.BinaryCrossentropy(),
        'binary_focal_crossentropy': tf.keras.losses.BinaryFocalCrossentropy(
            apply_class_balancing=hp.Choice("apply_class_balancing", [True,
                                                                      False]))
    }

    loss = loss_dict[loss_name]

    model.compile(
        optimizer=optimizer, 
        loss=loss,
        metrics=[tf.keras.metrics.BinaryAccuracy(),
                 tf.keras.metrics.AUC()]
    )
    return model

### Bayesian Optimization

In [ ]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    # 50 deneme yapılacak.
    max_trials=50,
    overwrite=True,
    # val_auc'u maksimize etmeye çalışalım.
    objective=keras_tuner.Objective("val_auc", direction="max"),
    directory="/tmp/keras_tuner_bayesian_results",
)

In [ ]:
tuner.search(
    train_ds,
    validation_data=valid_ds,
    epochs=100,
    callbacks=[tf.keras.callbacks.TensorBoard("/tmp/keras_tuner_bayesian_results"),
               tf.keras.callbacks.ReduceLROnPlateau(
                   monitor="val_loss",
                   patience=5,
                   verbose=1,
                   min_lr=1e-08,
                   factor = 0.1
               ),
               tf.keras.callbacks.EarlyStopping(
                   patience=18,
                   restore_best_weights=True,
                   verbose=1,
                   monitor="val_loss"
               )
               ],
)

Trial 50 Complete [00h 03m 13s]
val_auc: 0.7119433283805847

Best val_auc So Far: 0.727103054523468
Total elapsed time: 01h 19m 31s


Örnek çıktı:

```
Value             |Best Value So Far |Hyperparameter
5                 |5                 |mlp_layers
96                |96                |units_0
swish             |swish             |activation_dense
0.070805          |0.070805          |dropout_0
rmsprop           |rmsprop           |optimizer
0.0019492         |0.0019492         |learning_rate
binary_crossent...|binary_crossent...|loss
1                 |1                 |apply_class_balancing
```

## Tensorboard ile Sonuçlara Bakmak

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/keras_tuner_bayesian_results

## En İyi Modeli Almak

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hyperparameters

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(valid_ds)

79/79 [==============================] - 0s 3ms/step - loss: 0.1141 - binary_accuracy: 0.8131 - auc: 0.7271


[0.11408457905054092, 0.8131499886512756, 0.727103054523468]